# Content based method

# import libraries

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# read data

In [74]:
movies_df = pd.read_csv("movies.csv")
ratings_df = pd.read_csv("ratings.csv")

delete Nan if it was in dataset

In [75]:
movies_df = movies_df.dropna()
ratings_df = ratings_df.dropna()

In [76]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# Clean and prepare movies data

first we need to have a better movies_df, we need to delete year in title and also make genre 

so we use regex to detect 4 digits together in Parentheses sth like (1995)

In [77]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)

# here we replace digits of year with nothing
movies_df['title'] = movies_df.title.str.replace(r'\(\d{4}\)', '', regex=True)

# here we use strip because we need delete 
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


split generes column data based on '|' to make a list of data to work with

In [78]:
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


making a copy of movies_df to work with a alos to have movies_df without change

In [79]:
moviesWithGenres_df = movies_df.copy()

below we have index which represent the index of data and row represents every column in that row

In [80]:
for index, row in movies_df.iterrows():
    for genere in row['genres']:
        # here row['genres'] = list of genres
        # .at(row, column) -> so we are adding columns and so that they don't have any data we add it to 1
        moviesWithGenres_df.at[index, genere] = 1

# here we fill every column that was not 1 to 0        
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
moviesWithGenres_df.iloc[0]

movieId                                                               1
title                                                         Toy Story
genres                [Adventure, Animation, Children, Comedy, Fantasy]
year                                                               1995
Adventure                                                           1.0
Animation                                                           1.0
Children                                                            1.0
Comedy                                                              1.0
Fantasy                                                             1.0
Romance                                                             0.0
Drama                                                               0.0
Action                                                              0.0
Crime                                                               0.0
Thriller                                                        

here we can see toy story genre 

# prepare and clean ratings data

In [82]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [83]:
ratings_df = ratings_df.drop('timestamp', axis=1)
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


# User input

In [38]:
userInput = [
            {'title':'Dracula: Dead and Loving It', 'rating':3.5},
            {'title':'Toy Story', 'rating':4},
            {'title':'Assassins', 'rating':4},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Babe', 'rating':2}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,Dracula: Dead and Loving It,3.5
1,Toy Story,4.0
2,Assassins,4.0
3,Pulp Fiction,5.0
4,Babe,2.0


# getting the Movie ID

In [48]:
# here we filter titles -> we only keep titles that are in the movies list
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]

# here we combine movie and user input data
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies.drop('genres', axis=1).drop('year', axis=1)
inputMovies

,movieId,title,rating
0,1,Toy Story,4.0
1,12,Dracula: Dead and Loving It,3.5
2,23,Assassins,4.0
3,34,Babe,2.0
4,296,Pulp Fiction,5.0


# Add the user input to movie data frame we prepare before

In [49]:
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,12,Dracula: Dead and Loving It,"[Comedy, Horror]",1995,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,23,Assassins,"[Action, Crime, Thriller]",1995,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32,34,Babe,"[Children, Drama]",1995,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
257,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
# we have different index so we reset them to 0 to 5 for exmaple that have order
userMovies = userMovies.reset_index(drop=True)

# drop what we don't need
userGenreTable = userMovies.drop('movieId', axis=1).drop('title', axis=1).drop('genres', axis=1).drop('year', axis=1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Building base user profile

here we multipe the rating to genre of movies to get numbers below

In [51]:
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
userProfile

Adventure              4.0
Animation              4.0
Children               6.0
Comedy                12.5
Fantasy                4.0
Romance                0.0
Drama                  7.0
Action                 4.0
Crime                  9.0
Thriller               9.0
Horror                 3.5
Mystery                0.0
Sci-Fi                 0.0
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

### as we can see comedy was the favorite genre of the user

# Build Genre table 

In [55]:
# set index to movie ID
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])

genreTable = genreTable.drop('movieId', axis=1).drop('title', axis=1).drop('genres', axis=1).drop('year', axis=1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
genreTable.shape

(9742, 20)

here we multiple every movie of user that has 1 in its genre to rating and then we find sum of all for different genre and we divide it by all genere numbers and add all of different genre to get the result between 0 to 1 

In [56]:
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.484127
2    0.222222
3    0.198413
4    0.309524
5    0.198413
dtype: float64

In [59]:
# we use sort to have best recommendations on top
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)

recommendationTable_df.head()

movieId
81132    0.777778
4719     0.722222
7235     0.714286
5027     0.658730
5628     0.658730
dtype: float64

get the best recommendation by title

In [61]:
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,year
19,20,Money Train,"[Action, Comedy, Crime, Drama, Thriller]",1995
118,145,Bad Boys,"[Action, Comedy, Crime, Drama, Thriller]",1995
1103,1432,Metro,"[Action, Comedy, Crime, Drama, Thriller]",1997
2250,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
3460,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
3657,5027,Another 48 Hrs.,"[Action, Comedy, Crime, Drama, Thriller]",1990
3989,5628,Wasabi,"[Action, Comedy, Crime, Drama, Thriller]",2001
4409,6503,Charlie's Angels: Full Throttle,"[Action, Adventure, Comedy, Crime, Thriller]",2003
4693,7007,"Last Boy Scout, The","[Action, Comedy, Crime, Drama, Thriller]",1991
4843,7235,Ichi the Killer (Koroshiya 1),"[Action, Comedy, Crime, Drama, Horror, Thriller]",2001
